In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py as h5
from tqdm import tqdm
import warnings
import time
warnings.filterwarnings('ignore')

# Funciones para binnear

In [2]:
def initbin(numrows):

    '''
    calcula la cantidad de bines en base a la resolución espacial que se vaya a utilizar.
    input:
    numrows ---> Número de filas en la grilla. Este número depende de la resolución espacial que se vaya a emplear.

    output:
    totbins ---> Número posible de bines en la grilla
    basebin ---> Lista que contiene el número (identificador) del primer bin de cada fila. Con tamaño igual al número de filas
    numbin  ---> Lista que contiene el número total de bines posible en cada fila. Con tamaño igual al número de filas
    latbin  ---> Lista que contiene la latitud del primer bin de cada fila (centro del bin). Con tamaño igual al número de filas
    '''

    basebin = np.zeros((numrows,))
    latbin = np.zeros((numrows,))
    numbin = np.zeros((numrows,))
    basebin[0] = 1
    for row in range(numrows):
        latbin[row] = ((row + 0.5)*180.0/numrows) - 90.0
        numbin[row] = int(2*numrows*np.cos(latbin[row]*np.pi/180.0) + 0.5)
        if(row > 0):
            basebin[row] = basebin[row - 1] + numbin[row - 1]
    totbins = basebin[numrows - 1] + numbin[numrows - 1] - 1  
    return totbins, basebin, numbin, latbin

In [3]:
def lat2row(lat, numrows):

    '''
    Identifica la fila que le corresponde a una latitud dada
    Para calcular la fila que le corresponde a una cierta latitud, tomo esa latitud y le 
    sumo 90 (porque el rango en el que se mueven las latitudes es de -90 (Sur) a 90 (Norte))
    y lo multiplico por la cantidad total de las filas en la grilla dividido en 180 (90 del 
    hemisferio Sur) y (90 del hemisferio Norte)

    input:
    lat    ---> Latitud del pixel
    numrow ---> Número de filas en la grilla. Este número depende de la resolución espacial que se vaya a emplear.

    output:
    row    ---> Fila de la grilla donde cae el pixel 

    '''
    row = 0*lat
    row = (90 + lat)*numrows/180.0
    row = row.astype(int)
    row[row>=numrows] = numrows - 1
    return row

In [4]:
def constrain_lon(lon):

    '''
    Es un condicionante para las longitudes, si la longitud es mayor a 180
    o menor a -180 lo acomoda dentro del rango [-180, 180]

    input:
    lon  ---> longitud del pixel

    ouput:
    lon  ---> longitud del pixel reordenado si está fuera de rango
    '''
    lon[lon < -180] = lon[lon < -180] % 180
    lon[lon>180] = lon[lon>180] % -180
    # while(lon < -180):
    #     lon += 360
    # while(lon >  180):
    #     lon -= 360 
    return lon

In [5]:
def rowlon2bin(row, lon, numbin, basebin):
    
    '''
    Identifica el número del bin que corresponde a una cierta longitud 

    input:
    row     ---> Fila donde se encuenrta el pixel en base a su latitud
    lon     ---> longitud del pixel
    numbin  ---> número de bines en esa fila
    basebin ---> primer bin de esa fila

    output:
    binlon ---> número del bin

    '''
    lon = constrain_lon(lon)
    col = ((lon + 180.0)*numbin[row]/360.0)
    col = col.astype(int)
    mask = np.where(col >= numbin[row])
    col[mask] = numbin[row[mask]] - 1 
    # if(col >= numbin[row]):
    #     col = numbin[row] - 1

    return basebin[row] + col

In [6]:
def constrain_lat(lat):

    '''
    Es un condicionante para las latitudes, si la latitud es mayor a 90
    o menor a -90 lo acomoda dentro del rango [-90, 90]

    input:
    lat  ---> latitud del pixel

    ouput:
    lat  ---> latitud del pixel reordenado si está fuera de rango
    '''


    lat[lat>90] = 90
    lat[lat<-90] = -90
    # if(lat >  90):
    #     lat = 90
    # if(lat < -90): 
    #     lat = -90 
    return lat

In [7]:
def latlon2bin(numrow,lat,lon,numbin,basebin):

    '''
    Calcula el bin dada una latitud y longitud

    input:
    numrow  ---> Número de filas en la grilla. Este número depende de la resolución espacial que se vaya a emplear.
    lat     ---> latitud del pixel
    lon     ---> longitud del pixel
    numbin  ---> numero de bines en la fila
    basebin ---> primer bin de la fila

    ouput:
    binlon  ---> bin donde cae el pixel 
    '''

    lat = constrain_lat(lat)
    lon = constrain_lon(lon)

    row = lat2row(lat, numrow)
    binlon = rowlon2bin(row, lon, numbin, basebin)

    return binlon

In [8]:
# Falta vectorizar
def bin2latlon (_bin, numrows, basebin, numbin, latbin):

    
    '''

    La función bin2latlon calcula la latitud y la longitud dado un bin.
    La latitud y la longitud calculadas son del centro del bin
    Esta función necesita: el bin(identificador),el número de filas donde se ubica el bin, el primer bin(identificador) de la fila y la latitud del primer bin de la fila
    La latitud (central) del bin a calcular siempre sera la misma que la del primer bin de la fila, puesto que la latitud no cambia
    La longitud es calculada diviendo 360 en la cantidad de bines de la fila. Esto genera la distancia 
    entre cada bin. Este valor se multiplica por el bin analizado menos el primer bin de la fila y se 
    le suma 0.5 (para encontrar el centro del bin). Finalmente a esta operación se le resta 180 (para encontrar el hemismerio al que corresponde el bin)


    input:
    _bin    ---> Número del bin
    numrows ---> Número total de filas
    basebin ---> Número del primer bin de la fila
    numbin  ---> Cantidad de bines de la fila
    latbin  ---> latitud del primer bin de la fila ()

    ouput:
    clat  ---> Centro de la latitud del bin
    clon  ---> Centro de la longitud del bin
    '''

    row = numrows - 1
    if(_bin < 1):
        _bin = 1

    while(_bin < basebin[row]):
        row-=1

    clat = latbin[row]
    clon = 360.0*(_bin - basebin[row] + 0.5)/numbin[row] - 180.0
    
    return clat,clon

def bin2bounds(_bin, numrows, basebin, numbin, latbin):
    '''
    La función bin2bounds calcula los limites superior, inferior, derecho e izquierdo de cada bin
    Los limites superior e inferior de un bin siempre seran iguales para todos los bines de esa misma fila
    El limite superior (Norte) se calcula tomando la latitud central del primer bin de la fila analizada y se le suma 90 divido entre la cantidad de filas
    El limite inferior (Sur) se calcula tomando la latitud central del primer bin de la fila analizada y se le resta 90 divido entre la cantidad de filas
    El limite derecho (Oriente,East) se calcula tomando la longitud del bin analizado (centro) y se le suma 180 divido entre el número de bines de esa fila
    El limite izquierdo (Occidente,West) se calcula tomando la longitud del bin analizado (centro)y se le resta 180 divido entre el número de bines de esa fila

    input:
    _bin    ---> Número del bin
    numrows ---> Número total de filas
    basebin ---> Número del primer bin de la fila
    numbin  ---> Cantidad de bines de la fila
    latbin  ---> latitud del primer bin de la fila

    output:
    north ---> limite norte del bin
    south ---> limite sur del bin
    west  ---> limite oeste del bin
    east  ---> limite este del bin
    '''
    row = numrows - 1
    if(_bin < 1):
        _bin = 1

    while(_bin < basebin[row]):
        row-=1

    north = latbin[row] + 90.0/numrows
    south = latbin[row] - 90.0/numrows
    lon = 360.0*(_bin - basebin[row] + 0.5)/numbin[row] - 180.0
    west = lon - 180.0/numbin[row]
    east = lon + 180.0/numbin[row]
    return north,south,west,east

# Binneado

In [9]:
# def get_obs_data(ds, product_list, row_idx):
#     return np.array([ds['geophysical_data'][product_list[i]][row_idx,:] for i in range(len(product_list))]).T    

In [10]:
# %%time
# Nota: get_bin_index se corresponde con la funcion latlon2bin y devuelve el numero del bin al que corresponde dada lat, lon
# NSCANS = 3173  # es el nummero de filas del granulo
# NPIXEL = 1468 # es el numero de columnas del granulo

# def get_stats(NSCANS, NPIXEL, LAT, LON, numrow, numbin, basebin, ds, products):
#     for L in tqdm(range(NSCANS)):
#         for I in range(NPIXEL):
#             #if OBS
#             t1 
#             IDX = int(latlon2bin(numrow,LAT[L,I],LON[L,I],numbin,basebin))
#             OBS = get_obs_data(ds, products, L)
#             for J in range(NVARS):
#                 XLOG = np.log(OBS[I,J])
#                 # print(SUMX[IDX,J], XLOG)
#                 SUMX[IDX,J] += XLOG
#                 SUMXX[IDX,J] += XLOG*XLOG
#             N[IDX] = N[IDX] + 1
#             NSEG[IDX] = 1
            
#     SUMX2 = np.zeros((int(totbins), NVARS))
#     SUMXX2 = np.zeros((int(totbins), NVARS))
#     for IDX in tqdm(range(int(totbins))):
#         if N[IDX] > 0:
#             W[IDX] = np.sqrt(N[IDX])
#             for J in range(NVARS):
#                 SUMX2[IDX, J] = SUMX[IDX, J] / W[IDX]
#                 SUMXX2[IDX, J] = SUMXX[IDX, J] / W[IDX]
    
#     return SUMX, SUMXX, SUMX2, SUMXX2, N, W, NSEG

In [11]:
def get_spatial_bin_vectorized(NVARS, lat, lon, numrow, numbin, basebin, ds, products):

    '''
    Identifica todos los pixeles que caen dentro de un bin y calcula los estadisticos

    input:
    NVARS
    lat     ---> latitud del pixel
    lon     ---> longitud del pixel
    numrow  ---> Número de filas en la grilla. Este número depende de la resolución espacial que se vaya a emplear.
    numbin  ---> numero de bines en la fila
    basebin ---> primer bin de la fila
    ds
    products

    output:
    idx    ---> Lista que contiene los números de los bines
    XLOG   ---> Logaritmos de los productos asociado a un bin
    XXLOG  ---> Logaritmos de los productos al cuadrado de cada bin
    SUMX   ---> Suma de los logaritmos de las variables de los pixeles que caen dentro de un bin
    SUMXX  ---> Suma de los logaritmos al cuadrado de las variables de los pixeles que se encuentran en un mismo bin
    SUMXW  ---> Promedio de los logaritmos de las variables de los pixeles que se encuentran en un mismo bin
    SUMXXW ---> Promedio del cuadrado de los logaritmos de las variables de los pixeles que se encuentran en un mismo bin
    N      ---> Representa el numero de valores sumados dentro de SUMX y SUMXX para todas las variables y para idx (cantidad de pixeles que caen dentro de un bin)
    W      ---> Representa el peso para todos las variables para cada bin dado, calculado como la raiz cuadrada de N
    NSEG   ---> Represernta el número de escenas nivel 2
    '''
    SUMX = np.zeros((int(totbins), NVARS))
    SUMXX = np.zeros((int(totbins), NVARS))
    SUMXw = np.zeros((int(totbins), NVARS))
    SUMXXw = np.zeros((int(totbins), NVARS))
    N =  np.zeros((int(totbins), ))
    W =  np.zeros((int(totbins), ))
    NSEG = np.zeros((int(totbins), ))
    TT = np.zeros((int(totbins), ))
    
    idx = latlon2bin(numrow, lat, lon, numbin, basebin)
    idx = idx.astype(int)
    prod_stack = np.stack(ds['geophysical_data'][products[i]][:] for i in range(len(products)))
    prod_rolled = np.rollaxis(np.rollaxis(prod_stack, 1,0), 2,1)
    XLOG = np.log(prod_rolled)
    XXLOG = XLOG * XLOG
    
    unique, count = np.unique(idx, return_counts=True)

    for indice,count in zip(unique, count):
        m = np.where(idx == indice)
        coord = list(zip(m[0],m[1]))     
        
        for i in range(NVARS):
            for c in coord:
                if ~np.isnan(XLOG[c[0],c[1],i]):
                    SUMX[indice, i] = SUMX[indice, i] + XLOG[c[0], c[1], i]
                    
                if ~np.isnan(XXLOG[c[0],c[1],i]):
                    SUMXX[indice, i] = SUMXX[indice, i] + XXLOG[c[0], c[1], i]
            
        N[indice] = count
        W[indice] = np.sqrt(count)
        
        SUMXw[indice] = SUMX[indice] / W[indice]
        SUMXXw[indice]= SUMXX[indice]/W[indice]
    
    
    return idx, XLOG,  SUMX, SUMXX, SUMXw, SUMXXw, N, W, NSEG      

# Binneando un L2 de MODIS

## Exploremos el netCDF de MODIS

In [12]:
file = './A2022160171000.L2_LAC_OC.nc'
l2modis = h5.File(file,'r')
l2modis.keys()

<KeysViewHDF5 ['bands_per_pixel', 'geophysical_data', 'navigation_data', 'number_of_bands', 'number_of_lines', 'number_of_reflective_bands', 'pixel_control_points', 'pixels_per_line', 'processing_control', 'scan_line_attributes', 'sensor_band_parameters']>

In [13]:
l2modis['navigation_data'].keys()

<KeysViewHDF5 ['longitude', 'latitude', 'cntl_pt_cols', 'cntl_pt_rows', 'tilt']>

In [14]:
modisprod = list(l2modis['geophysical_data'].keys())[:-1]
l2modis['geophysical_data'].keys()

<KeysViewHDF5 ['aot_869', 'angstrom', 'Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488', 'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645', 'Rrs_667', 'Rrs_678', 'chlor_a', 'chl_ocx', 'Kd_490', 'pic', 'poc', 'ipar', 'nflh', 'par', 'l2_flags']>

In [15]:
l2modis['pixels_per_line'][:].size

1354

In [16]:
l2modis['navigation_data']['longitude'].shape

(2030, 1354)

In [17]:
numrow = 4320

LAT = l2modis['navigation_data']['latitude'][:]
LON = l2modis['navigation_data']['longitude'][:]

In [18]:
lat2row(LAT, numrow)

array([[  1265,   1264,   1264, ...,   1171,   1171,   1171],
       [  1265,   1265,   1264, ...,   1171,   1171,   1171],
       [  1265,   1265,   1265, ...,   1172,   1172,   1172],
       ...,
       [-21816, -21816, -21816, ..., -21816, -21816, -21816],
       [-21816, -21816, -21816, ..., -21816, -21816, -21816],
       [-21816, -21816, -21816, ..., -21816, -21816, -21816]])

In [19]:
(90 + LAT)*numrow/180.0

array([[  1265.1213,   1264.782 ,   1264.4459, ...,   1171.4861,
          1171.4471,   1171.4087],
       [  1265.5377,   1265.1971,   1264.8599, ...,   1171.9253,
          1171.888 ,   1171.8508],
       [  1265.9542,   1265.6125,   1265.2739, ...,   1172.3647,
          1172.3286,   1172.293 ],
       ...,
       [-21816.    , -21816.    , -21816.    , ..., -21816.    ,
        -21816.    , -21816.    ],
       [-21816.    , -21816.    , -21816.    , ..., -21816.    ,
        -21816.    , -21816.    ],
       [-21816.    , -21816.    , -21816.    , ..., -21816.    ,
        -21816.    , -21816.    ]], dtype=float32)

In [20]:
LAT

array([[ -37.28661 ,  -37.30075 ,  -37.314754, ...,  -41.18808 ,
         -41.1897  ,  -41.191303],
       [ -37.26926 ,  -37.28345 ,  -37.29751 , ...,  -41.169777,
         -41.171337,  -41.172882],
       [ -37.251907,  -37.266148,  -37.280254, ...,  -41.15147 ,
         -41.152977,  -41.15446 ],
       ...,
       [-999.      , -999.      , -999.      , ..., -999.      ,
        -999.      , -999.      ],
       [-999.      , -999.      , -999.      , ..., -999.      ,
        -999.      , -999.      ],
       [-999.      , -999.      , -999.      , ..., -999.      ,
        -999.      , -999.      ]], dtype=float32)

## Calculamos el binneado y los estadísticos

In [21]:
numrow = 4320
totbins, basebin, numbin, latbin = initbin(numrow)

In [22]:
%%time
LAT = l2modis['navigation_data']['latitude'][:]
LON = l2modis['navigation_data']['longitude'][:]
NVARS = len(modisprod)
idx, XLOG,  SUMX, SUMXX, SUMXw, SUMXXw, N, W, NSEG = get_spatial_bin_vectorized(NVARS, LAT, LON, numrow, numbin, basebin, l2modis, modisprod)

CPU times: user 41min 1s, sys: 1.7 s, total: 41min 3s
Wall time: 41min 16s


In [23]:
SUMX.shape

(23761676, 20)

In [24]:
c = SUMX[:, 12]